In [27]:
from __future__ import print_function
import argparse
import numpy as np
import PIL
from PIL import Image, ImageChops
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import copy
import requests
from io import BytesIO
import math
import keyboard
import random
import os
from pathlib import Path

directory = './coco128/images' # директория для поиска имён файлов изображений

# извлечение имён файлов изображений тренировочного набора
train_images = []
for root, dirs, files in os.walk(directory):  
    for filename in files:
        train_images.append(filename)

full_directory = directory + '/train2017/' # директория для извлечения ключевых точек из изображений

# Шаг 2. Извлечение ключевых точек ( нужны только дескрипторы )
def ExtractKeyPoints():
    all_descriptors = []
    for filename in train_images:

        # считывание изображения и преобразование его в оттенки серого
        img = cv2.imread(full_directory + filename)
        gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # нахождение ключевых точек и дескрипторов изображения
        sift = cv2.SIFT_create()
        kp = sift.detect(gray, None)
        
        # добавление дескрипторов в массив
        for d in kp:
            all_descriptors.append(d.pt)
        
    return all_descriptors

In [28]:
#all_descriptors = ExtractKeyPoints()

from sklearn.cluster import KMeans
import pickle
from joblib import dump, load

# Шаг 3. Обучение модели кластеризации
K = 1024
#kmeans = KMeans(n_clusters=K, random_state=0, n_init="auto").fit(all_descriptors)

filename_model_claster = 'model_clasteris.joblib'

#сохранение обученной модели
#dump(kmeans, filename_model_claster)

#загрузка существующей обученной модели
kmeans = load(filename_model_claster)


In [29]:
#labels = kmeans.labels_ # метки кластеров для каждого дескриптора (какому классу или кластеру дескриптор принадлежит )
#cluster_centers = kmeans.cluster_centers_ # центры кластеров (классов)


# Шаг 4. Представление изображения в виде вектора в метрическом пространстве
def ConvertImageToVector(img):
    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # нахождение дескрипторов изображения
    sift = cv2.SIFT_create()
    kp = sift.detect(gray, None)
    des = [d.pt for d in kp]

    # нахождение меток дескрипторов данного изображения (какому классу или кластеру они принадлежат)
    prediction = kmeans.predict(des)

    # создание гистограммы частоты вхождения каждого класса в данном изображении - результирующего вектора
    vector, _ = np.histogram(prediction, bins=K)

    return vector
    

In [31]:
# Пример
img = cv2.imread(full_directory + '000000000030.jpg')
v = ConvertImageToVector(img)

v

array([1, 0, 0, ..., 2, 0, 9], dtype=int64)